In [1]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


class Function:
    def __call__(self, input):
        # save the input variable
        self.input = input
        
        x = input.data
        
        # exact calculation is held at forward function
        y = self.forward(x)
        
        output = Variable(y)
        
        # connect the relation between the Variable and Function
        output.set_creator(self)
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):
        raise NotImplementedError()

        
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [3]:
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [4]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx        

# Usage improve

## 1. functional

we used function like this

In [5]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

let's make this more easy

In [6]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

we could make this one line

In [7]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

**This kind of thing is created every time so this should not have weight data on it.**

In [8]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

# backpropation
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


By making functional we can use it consecutive

In [10]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

# backpropation
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## 2. `backward` simplify

remove the needs to right 

```python
y.grad = np.array(1.0)
```

In [11]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


In [16]:
print(np.ones_like(7))
print(np.ones_like([7, 7]))
print(np.ones_like([[7, 7],
                    [7, 7]]))

1
[1 1]
[[1 1]
 [1 1]]


In [17]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

# backpropation
y.backward()
print(x.grad)

3.297442541400256


# Protection add

## numpy only

In [18]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')

        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [19]:
x = Variable(np.array(1.0))
x = Variable(None)

x = Variable(1.0)

TypeError: <class 'float'> is not supported

But there is a small problem here

In [20]:
x = np.array([1.0])
y = x ** 2

print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


When we start numpy with `0-dim` it outputs type `numpy.float`

In [21]:
x = np.array(1.0)
y = x ** 2

print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


we need this to change to `ndarray`!

In [22]:
def as_array(x):
    if np.isscaler(x):
        return np.array(x)
    return x

In [23]:
np.isscalar(np.float64(1.0))

True

In [24]:
np.isscalar(2.0)

True

In [25]:
np.isscalar(np.array(1.0))

False

In [26]:
np.isscalar(np.array([1, 2, 3]))

False

In [ ]:
class Function:
    def __call__(self, input):
        # save the input variable
        self.input = input
        
        x = input.data
        
        # exact calculation is held at forward function
        y = self.forward(x)
        
        output = Variable(as_array(y))
        
        # connect the relation between the Variable and Function
        output.set_creator(self)
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):
        raise NotImplementedError()